In [19]:
model_file = "../results/"
queries_file = "../results/"
results_file = "../results/"

with open("../data/busca.cfg", "r") as config_file:
    for line in config_file.readlines():
        instruction, filename = line.split("=")
        filename = filename.strip()
        
        if instruction == "MODELO":
            model_file += filename
        elif instruction == "CONSULTAS":
            queries_file += filename
        elif instruction == "RESULTADOS":
            results_file += filename

In [20]:
import pandas as pd

model = pd.read_csv(model_file, sep=";")
model.set_index(["Token"], inplace=True)

In [21]:
import generate_inverted_list as gil

queries = pd.read_csv(queries_file, sep=";")
queries.set_index(["QueryNumber"], inplace=True)
for number, text in queries.itertuples():
    processed_text = gil.preproccess_text(text)
    queries.at[number, "QueryText"] = processed_text

In [24]:
# Add queries no modelo
import numpy as np

shape = (model.shape[0], 1)
for qnumber, qtext in queries.itertuples():
    zeros = pd.DataFrame(np.zeros(shape), index=model.index, columns=[f"Q{qnumber}"])
    model = pd.concat([model, zeros], axis=1)
    for word in qtext:
        if not word in model.index:
            zeros = pd.DataFrame(np.zeros((1, len(model.columns))), index=[word], columns=model.columns)
            model = pd.concat([model, zeros], axis=0)
            shape = (model.shape[0], 1)
        model.at[word, f"Q{qnumber}"] += 1

In [6]:
def get_vector(document, model):
    return model[str(document)].to_numpy()  

In [7]:
def get_vector_size(vector):
    return np.linalg.norm(vector)

In [8]:
def sim_cos(query, document, model):
    q = get_vector(query, model)
    d = get_vector(document, model)

    q_dot_d = np.dot(q, d)
    qxd = get_vector_size(q) * get_vector_size(d)
    
    return q_dot_d / qxd

In [27]:
ranking = pd.DataFrame()
shape = (model.shape[1] - 99, 1)
for query in queries.index:
    q = f"Q{query}"
    zeros = pd.DataFrame(np.zeros(shape), index=model.columns[:-99], columns=[q])
    ranking = pd.concat([ranking, zeros], axis=1)
    # Iterar pelas colunas de documento, retirando as de query
    for document in model.columns[:-99]:
        result = sim_cos(q, document, model)
        ranking.loc[document, q] = result

In [29]:
ranking

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q90,Q91,Q92,Q94,Q95,Q96,Q97,Q98,Q99,Q100
1,0.017383,0.184724,0.017383,0.086635,0.000000,0.012291,0.029738,0.070738,0.000000,0.021289,...,0.154672,0.019179,0.024583,0.0,0.016093,0.015054,0.016093,0.023222,0.024583,0.019042
6,0.018739,0.123805,0.018739,0.049812,0.000000,0.013251,0.016229,0.040671,0.000000,0.022951,...,0.136055,0.000000,0.026501,0.0,0.017349,0.016229,0.222084,0.000000,0.026501,0.020528
19,0.029854,0.000000,0.084833,0.000000,0.000000,0.021110,0.073468,0.041421,0.000000,0.074193,...,0.025854,0.000000,0.042220,0.0,0.036669,0.025854,0.027640,0.000000,0.042220,0.047745
24,0.023859,0.000000,0.051974,0.000000,0.020955,0.067810,0.045011,0.014571,0.000000,0.029221,...,0.020662,0.065809,0.033741,0.0,0.058058,0.020662,0.033290,0.024768,0.050851,0.026136
30,0.015868,0.000000,0.023621,0.000000,0.000000,0.011220,0.031180,0.007753,0.000000,0.019434,...,0.013742,0.000000,0.022441,0.0,0.240507,0.031098,0.030955,0.000000,0.022441,0.017382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,0.000000,0.050015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.032891,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
908,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
117,0.000000,0.000000,0.000000,0.000000,0.138831,0.000000,0.000000,0.000000,0.030691,0.000000,...,0.000000,0.000000,0.000000,0.0,0.074208,0.000000,0.074208,0.000000,0.113355,0.000000
1011,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040709,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.033012


In [13]:
def get_results(file, ranking):
    with open(file, 'w') as results:
        results.write("QueryNumber;DocInfos\n")
        for query in ranking.columns:
            query_number = query.replace('Q', '')
            sorted_ranking = ranking[query].sort_values(ascending=False)
            position_ranking = 1
            for doc_number, cos in sorted_ranking.items():
                if cos == 0:
                    break
                doc_infos = [position_ranking, doc_number, cos]
                position_ranking += 1
                results.write(f"{query_number};{doc_infos}\n")

In [14]:
get_results(results_file, ranking)